In [1]:
import os
import time

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
DATA_PATH = 'data'
train_path = os.path.join(DATA_PATH, 'train')
val_path = os.path.join(DATA_PATH, 'val')
test_path = os.path.join(DATA_PATH, 'test')

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_dataset = datasets.ImageFolder(train_path, transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]))
valid_dataset = datasets.ImageFolder(val_path, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))
print(len(train_dataset))

6516


In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    num_workers=4
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=32,
    num_workers=4
)

In [5]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)     
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)     
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4_bn = nn.BatchNorm2d(128)
        
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv6_bn = nn.BatchNorm2d(512)
        
        self.conv7 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv8_bn = nn.BatchNorm2d(512)
        
#         self.conv9 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
#         self.conv10 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
#         self.conv10_bn = nn.BatchNorm2d(512)
        
        self.fc1 = nn.Linear(512 * 14 * 14, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(F.relu(self.conv2_bn(self.conv2(x))))
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(F.relu(self.conv4_bn(self.conv4(x))))
        
        x = self.conv5(x)
        x = F.relu(x)
        x = self.pool(F.relu(self.conv6_bn(self.conv6(x))))
    
        x = self.conv7(x)
        x = F.relu(x)
        x = self.pool(F.relu(self.conv8_bn(self.conv8(x))))
    
#         x = self.conv9(x)
#         x = F.relu(x)
#         x = self.pool(F.relu(self.conv10_bn(self.conv10(x))))
        
        x = x.view(-1, 512*14*14)
        x = F.dropout(F.relu(self.fc1(x)), training=self.training, p=0.4)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [7]:
from torchsummary import summary
model = Net(196)
model.to(device)

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             448
            Conv2d-2         [-1, 32, 224, 224]           4,640
       BatchNorm2d-3         [-1, 32, 224, 224]              64
         MaxPool2d-4         [-1, 32, 112, 112]               0
            Conv2d-5         [-1, 64, 112, 112]          18,496
            Conv2d-6        [-1, 128, 112, 112]          73,856
       BatchNorm2d-7        [-1, 128, 112, 112]             256
         MaxPool2d-8          [-1, 128, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         295,168
           Conv2d-10          [-1, 512, 56, 56]       1,180,160
      BatchNorm2d-11          [-1, 512, 56, 56]           1,024
        MaxPool2d-12          [-1, 512, 28, 28]               0
           Conv2d-13          [-1, 512, 28, 28]       2,359,808
           Conv2d-14          [-1, 512,

/home/xiaolong/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum=0.9)

In [9]:
def train(model,device, train_loader, epoch):
    model.train()
    for data in tqdm(train_loader):
        x,y= data
        x=x.to(device)
        y=y.to(device)
        optimizer.zero_grad()
        y_hat= model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
    print ('Train Epoch: {}\t Loss: {:.6f}'.format(epoch,loss.item()))

In [10]:
def valid(model, device, valid_loader):
    model.eval()
    valid_loss = 0
    correct = 0
    with torch.no_grad():
        for data in tqdm(valid_loader):          
            x,y= data
            x=x.to(device)
            y=y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            valid_loss += criterion(y_hat, y).item() # sum up batch loss
            pred = y_hat.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
    valid_loss /= len(valid_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        valid_loss, correct, len(valid_dataset), 100. * correct / len(valid_dataset)))

In [11]:
for epoch in range(1, 20):
    train(model=model, device=device, train_loader=train_loader, epoch=epoch)
    valid(model=model, device=device, valid_loader=valid_loader)

  0%|          | 0/51 [00:00<?, ?it/s]

Train Epoch: 1	 Loss: 5.361686


  0%|          | 0/204 [00:00<?, ?it/s]


Test set: Average loss: 0.1654, Accuracy: 9/1628 (1%)



  0%|          | 0/51 [00:00<?, ?it/s]

...


Test set: Average loss: 0.1662, Accuracy: 9/1628 (1%)



  0%|          | 0/51 [00:00<?, ?it/s]

Train Epoch: 19	 Loss: 5.069221


100%|██████████| 51/51 [00:06<00:00,  7.37it/s]


Test set: Average loss: 0.1662, Accuracy: 9/1628 (1%)



In [ ]:
1/196 vs 9/1628